In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [2]:
t_bat19=pd.read_csv("C:/Users/a/Desktop/2020빅콘/제공데이터/제공데이터(.CSV)_시즌별, 시트별 구분/2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv",encoding='utf-8',engine='python')

In [85]:
#타율추가
t_bat19['HIT_RATE']=t_bat19['HIT']/t_bat19['AB']

## 한팀에 대해서(LG)

In [86]:
#LG 경기
LG=t_bat19[t_bat19['T_ID']=='LG']

In [87]:
LG.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,...,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,HIT_RATE
3,20190323LGHT0,20190323,LG,HT,0,T,34,31,2,2,...,0,0,12,3,0,5,0.142857,7,1,0.258065
13,20190324LGHT0,20190324,LG,HT,0,T,46,40,8,9,...,0,0,5,0,0,10,0.272727,11,3,0.300000
23,20190326LGSK0,20190326,LG,SK,0,T,38,31,6,6,...,0,1,2,0,0,5,0.285714,7,2,0.161290
33,20190327LGSK0,20190327,LG,SK,0,T,46,40,0,1,...,0,0,10,2,1,12,0.166667,12,2,0.225000
43,20190328LGSK0,20190328,LG,SK,0,T,30,29,1,1,...,0,0,9,0,0,2,0.000000,0,0,0.068966


In [88]:
#G_ID,날짜,팀명 제거
LG=LG.drop(LG.columns[[0,1,2]],axis='columns')
LG.head()

,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,HIT,H2,H3,...,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,HIT_RATE
3,HT,0,T,34,31,2,2,8,3,0,...,0,0,12,3,0,5,0.142857,7,1,0.258065
13,HT,0,T,46,40,8,9,12,3,0,...,0,0,5,0,0,10,0.272727,11,3,0.300000
23,SK,0,T,38,31,6,6,5,2,0,...,0,1,2,0,0,5,0.285714,7,2,0.161290
33,SK,0,T,46,40,0,1,9,1,0,...,0,0,10,2,1,12,0.166667,12,2,0.225000
43,SK,0,T,30,29,1,1,2,0,0,...,0,0,9,0,0,2,0.000000,0,0,0.068966


In [89]:
print('70%:',len(LG)*0.7,' ,30%:',len(LG)*0.3)

70%: 100.8  ,30%: 43.199999999999996


In [90]:
#데이터 셋 나누기
train_df=LG[:100]
test_df=LG[101:]
print(len(train_df),len(test_df))

100 43


In [91]:
##########데이터 전처리
x_train = train_df.drop(['HIT_RATE'], axis=1)
x_test = test_df.drop(['HIT_RATE'], axis=1)
y_train = train_df['HIT_RATE']
y_test = test_df['HIT_RATE']

In [92]:
x_train.head()

,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,HIT,H2,H3,...,BB,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN
3,HT,0,T,34,31,2,2,8,3,0,...,3,0,0,12,3,0,5,0.142857,7,1
13,HT,0,T,46,40,8,9,12,3,0,...,6,0,0,5,0,0,10,0.272727,11,3
23,SK,0,T,38,31,6,6,5,2,0,...,5,0,1,2,0,0,5,0.285714,7,2
33,SK,0,T,46,40,0,1,9,1,0,...,6,0,0,10,2,1,12,0.166667,12,2
43,SK,0,T,30,29,1,1,2,0,0,...,1,0,0,9,0,0,2,0.000000,0,0


In [93]:
x_train.columns

Index(['VS_T_ID', 'HEADER_NO', 'TB_SC', 'PA', 'AB', 'RBI', 'RUN', 'HIT', 'H2',
       'H3', 'HR', 'SB', 'CS', 'SH', 'SF', 'BB', 'IB', 'HP', 'KK', 'GD', 'ERR',
       'LOB', 'P_HRA_RT', 'P_AB_CN', 'P_HIT_CN'],
      dtype='object')

ANN(인공신경망) 돌릴 시 범주형 변수는 더미변수로 전환해주어야 한다.

해당 데이터는 'VS_T_ID'(상대팀명), 'TB_SC'(초말) column이 범주형으로 더미변수로 전환이 필요

In [94]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['VS_T_ID', 'TB_SC']),remainder='passthrough')
transformer.fit(x_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['VS_T_ID', 'TB_SC'])])

In [95]:
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [96]:
##########모델 생성

model = RandomForestRegressor()

In [97]:
##########모델 학습

model.fit(x_train, y_train)

RandomForestRegressor()

In [106]:
##########모델 검증
print(model.score(x_train, y_train)) 
print(model.score(x_test, y_test))

0.9921984217638133
0.9356063098384754


In [105]:
y_predict = model.predict(x_test)
print(y_predict[0]) 

0.24600441259068329


## 전체 팀에 대하여

In [107]:
t_bat19.head()

,G_ID,GDAY_DS,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,...,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,HIT_RATE
0,20190323HHOB0,20190323,HH,OB,0,T,42,34,4,4,...,0,1,7,2,1,11,0.200000,10,2,0.352941
1,20190323LGHT0,20190323,HT,LG,0,B,33,28,0,0,...,0,1,6,1,0,6,0.000000,9,0,0.107143
2,20190323KTSK0,20190323,KT,SK,0,T,39,36,4,4,...,0,0,10,1,1,8,0.200000,10,2,0.250000
3,20190323LGHT0,20190323,LG,HT,0,T,34,31,2,2,...,0,0,12,3,0,5,0.142857,7,1,0.258065
4,20190323WOLT0,20190323,LT,WO,0,B,37,35,4,4,...,0,1,11,0,2,6,0.230769,13,3,0.200000


In [112]:
#G_ID,날짜 제거
t_bat19=t_bat19.drop(t_bat19.columns[[0,1]],axis='columns')
t_bat19.head()

,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,HIT,H2,...,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN,HIT_RATE
0,HH,OB,0,T,42,34,4,4,12,1,...,0,1,7,2,1,11,0.200000,10,2,0.352941
1,HT,LG,0,B,33,28,0,0,3,1,...,0,1,6,1,0,6,0.000000,9,0,0.107143
2,KT,SK,0,T,39,36,4,4,9,0,...,0,0,10,1,1,8,0.200000,10,2,0.250000
3,LG,HT,0,T,34,31,2,2,8,3,...,0,0,12,3,0,5,0.142857,7,1,0.258065
4,LT,WO,0,B,37,35,4,4,7,2,...,0,1,11,0,2,6,0.230769,13,3,0.200000


In [113]:
print('70%:',len(t_bat19)*0.7,' ,30%:',len(t_bat19)*0.3)

70%: 1007.9999999999999  ,30%: 432.0


In [114]:
#데이터 셋 나누기
train_df=t_bat19[:1008]
test_df=t_bat19[1008:]
print(len(train_df),len(test_df))

1008 432


In [115]:
##########데이터 전처리
x_train = train_df.drop(['HIT_RATE'], axis=1)
x_test = test_df.drop(['HIT_RATE'], axis=1)
y_train = train_df['HIT_RATE']
y_test = test_df['HIT_RATE']

In [116]:
x_train.head()

,T_ID,VS_T_ID,HEADER_NO,TB_SC,PA,AB,RBI,RUN,HIT,H2,...,BB,IB,HP,KK,GD,ERR,LOB,P_HRA_RT,P_AB_CN,P_HIT_CN
0,HH,OB,0,T,42,34,4,4,12,1,...,5,0,1,7,2,1,11,0.200000,10,2
1,HT,LG,0,B,33,28,0,0,3,1,...,3,0,1,6,1,0,6,0.000000,9,0
2,KT,SK,0,T,39,36,4,4,9,0,...,3,0,0,10,1,1,8,0.200000,10,2
3,LG,HT,0,T,34,31,2,2,8,3,...,3,0,0,12,3,0,5,0.142857,7,1
4,LT,WO,0,B,37,35,4,4,7,2,...,1,0,1,11,0,2,6,0.230769,13,3


In [117]:
transformer = make_column_transformer(
    (OneHotEncoder(), ['T_ID', 'VS_T_ID', 'TB_SC']),remainder='passthrough')
transformer.fit(x_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(),
                                 ['T_ID', 'VS_T_ID', 'TB_SC'])])

In [118]:
x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

In [140]:
##########모델 생성

model = RandomForestRegressor()

In [120]:
##########모델 학습

model.fit(x_train, y_train)

RandomForestRegressor()

In [121]:
##########모델 검증
print(model.score(x_train, y_train)) 
print(model.score(x_test, y_test))

0.9987930719498931
0.9938970881230369


랜덤 포레스트 장점 : 변수 중요도 추출 가능

In [138]:
importance(model)
#varImpPlot(model, main="varImpPlot of iris")

NameError: name 'importance' is not defined

In [136]:
MSE<-rep(NA,6)
set.seed(1)
rf.boston5<-randomForest(HIT_RATE~.,data=t_bat19, mtry=5, importance=TRUE, do.trace=500)

SyntaxError: invalid syntax (<ipython-input-136-ad8520b15f7d>, line 3)

In [ ]:
rf.boston6<-randomForest(HIT_RATE~.,data=t_bat19, mtry=6, importance=TRUE, do.trace=500)

In [ ]:
rf.boston7<-randomForest(HIT_RATE~.,data=t_bat19, mtry=7, importance=TRUE, do.trace=500)